1 men 2 women
compare lines to see if same name with 1 and 2 = unisex 
get nombre
get total nombre de naissance pour cette année 


In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

In [2]:
df = pd.read_csv("data/dpt2020.csv", sep=";")
df['annais'] = pd.to_numeric(df['annais'], errors='coerce')
df = df.dropna(subset=['annais'])
df['annais'] = df['annais'].astype(int)

In [3]:

# Group by name and sex and sum 'nombre' across all departement
# grouped = df_year.groupby(['preusuel', 'sexe'])['nombre'].sum().reset_index()
grouped = df.groupby(['preusuel', 'sexe', 'annais'])['nombre'].sum().reset_index()

# we get a table with only one line for each name and sex with the number of births summed

In [4]:
grouped.head(5)

,preusuel,sexe,annais,nombre
0,AADIL,1,1983,3
1,AADIL,1,1992,3
2,AAHIL,1,2016,3
3,AALIYA,2,2017,3
4,AALIYAH,2,2001,9


In [5]:
# to find unisex names, i group by name and count number of sexes per name (either 1, 2 or both)
# name_sex_count = grouped.groupby('preusuel')['sexe'].nunique().reset_index()
name_sex_count = grouped.groupby(['preusuel', 'annais'])['sexe'].nunique().reset_index()

name_sex_count


,preusuel,annais,sexe
0,AADIL,1983,1
1,AADIL,1992,1
2,AAHIL,2016,1
3,AALIYA,2017,1
4,AALIYAH,2001,1
...,...,...,...
249242,ÖMER,2016,1
249243,ÖMER,2017,1
249244,ÖMER,2018,1
249245,ÖMER,2019,1


In [6]:
# keep the names where sexes is 2 (both men and women name)
# unisex_names = name_sex_count[name_sex_count['sexe'] == 2]['preusuel']
unisex_name = name_sex_count[name_sex_count['sexe'] == 2]


In [7]:
# Filter only unisex names
# df_unisex = grouped[grouped['preusuel'].isin(unisex_names)]
df_unisex = grouped.merge(unisex_name[['preusuel', 'annais']], on=['preusuel', 'annais'])


In [8]:

df_plot = df_unisex.pivot(index=['preusuel', 'annais'], columns='sexe', values='nombre').fillna(0).reset_index()
df_plot.columns = ['preusuel', 'annais', 'men', 'women']
df_plot['total'] = df_plot['men'] + df_plot['women']


In [9]:

# melt to get right formatting
# df_plot = pivot_df.reset_index().melt(id_vars=['preusuel', 'total'], value_vars=['men', 'women'], var_name='sex', value_name='count')

df_plot = df_plot.melt(id_vars=['preusuel', 'annais', 'total'],
                       value_vars=['men', 'women'],
                       var_name='sex', value_name='count')

df_plot.head(5)

,preusuel,annais,total,sex,count
0,ABDON,1918,8,men,5
1,ABDON,1922,19,men,15
2,ABDON,1927,20,men,11
3,ABDON,1935,14,men,11
4,ABDON,1937,6,men,3


In [10]:
# Compute % for each name/sex/year
df_plot['pct'] = df_plot['count'] / df_plot['total']
df_plot

,preusuel,annais,total,sex,count,pct
0,ABDON,1918,8,men,5,0.625000
1,ABDON,1922,19,men,15,0.789474
2,ABDON,1927,20,men,11,0.550000
3,ABDON,1935,14,men,11,0.785714
4,ABDON,1937,6,men,3,0.500000
...,...,...,...,...,...,...
16677,ÉDEN,2020,228,women,13,0.057018
16678,ÉLIE,2015,73,women,4,0.054795
16679,ÉLIE,2017,100,women,3,0.030000
16680,ÉLIE,2019,107,women,6,0.056075


In [11]:
# Slider
year_slider = alt.binding_range(min=int(df_plot['annais'].min()), max=int(df_plot['annais'].max()), step=1)


In [12]:
year_select = alt.selection_point(
    name="Select",
    fields=['annais'],
    bind=year_slider
)

chart = alt.Chart(df_plot).transform_filter(
    year_select
).transform_window(
    rank='rank()',
    sort=[alt.SortField('total', order='descending')],
    groupby=['annais']
).transform_filter(
    (alt.datum.rank <= 10)
).mark_bar().encode(
    x=alt.X('pct:Q', stack='normalize', axis=alt.Axis(format='%'), title='Percentage'),
    y=alt.Y('preusuel:N', sort='-x'),  # Sort by % descending
    color=alt.Color('sex:N', scale=alt.Scale(domain=['men', 'women'], range=['purple', 'red'])),
    tooltip=['preusuel', 'sex', alt.Tooltip('pct:Q', format='.0%'), 'total']
).add_params(
    year_select
).properties(
    width=600,
    height=400,
    title='Top 10 unisex names — % men/women — Select year'
)

chart


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

# NEW

In [13]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [14]:
# Pivot per name/year to get counts per sex
pivot_year = grouped.pivot(index=['preusuel', 'annais'], columns='sexe', values='nombre').fillna(0)
pivot_year.columns = ['men', 'women']  #sexe 1=men, 2=women

# Compute %
pivot_year['pct_male'] = pivot_year['men'] / (pivot_year['men'] + pivot_year['women'])

df_pct = pivot_year.reset_index()


In [15]:
# name_select = alt.selection_point(
#     name="name_select",
#     fields=["preusuel"],
#     bind=alt.binding_select(
#         options=sorted(df_pct['preusuel'].unique()),
#         name='Name: '
#     ),
#     value='CAMILLE',
#     clear=False,  # Keep selection when clicking
#     on="click"    # Also listen for clicks
# )

df_plot = df_plot[df_plot['preusuel'] != '_PRENOMS_RARES']


click_name = alt.selection_point(
    name="click_name",
    fields=["preusuel"],
    on="click",
    value="CAMILLE",
    
)

# name_select = alt.selection_point(
#     name="name_select",
#     fields=["preusuel"],
#     bind=alt.binding_select(
#         options=sorted(df_pct['preusuel'].unique().tolist()),
#         name='Name: '
#     ),
#     value='CAMILLE',
# )

# year_slider = alt.binding_range(min=int(df_plot['annais'].min()), max=int(df_plot['annais'].max()), step=1)

year_slider = alt.binding_range(
    name='Year:', 
    min=int(df_plot['annais'].min()), max=int(df_plot['annais'].max()), step=1)


year_select = alt.selection_point(
    name="Select",
    fields=['annais'],
    bind=year_slider,
    value=2000
)

click = alt.selection_point(
    fields=['preusuel', 'annais'],
    on='click',
    empty=True,
    clear='dblclick'
)


chart = alt.Chart(df_plot).transform_filter(
    year_select
).transform_window(
    rank='rank()',
    sort=[alt.SortField('total', order='descending')],
    groupby=['annais']
).transform_filter(
    (alt.datum.rank <= 10) # it will output the top 5 since each name has 2 lines (men and women)
).mark_bar().encode(
    x=alt.X('pct:Q', stack='normalize', axis=alt.Axis(format='%'), title='Percentage'),
    y=alt.Y('preusuel:N', sort='-x'),
    color=alt.Color('sex:N', scale=alt.Scale(domain=['men', 'women'], range=['purple', 'red'])),
    tooltip=['preusuel', 'sex', alt.Tooltip('pct:Q', format='.0%'), 'total'],
    opacity=alt.condition(click_name, alt.value(1), alt.value(0.4))  # Highlight selected bar
).add_params(
    year_select,
    click_name
).properties(
    width=300,
    height=150,
    title='Top 5 unisex names for selected year'
)




In [16]:
# names to plot
names_to_plot = ['CAMILLE', 'ALEX']

df_pct_plot = df_pct[df_pct['preusuel'].isin(names_to_plot)]

In [7]:

years_to_show = list(range(1900, 2021, 5))

all_years = pd.DataFrame({'annais': years_to_show})

df_full = all_years.merge(df_pct, on='annais', how='left')

labels = alt.Chart(pd.DataFrame({
    'y': [0, 1],
    'label': ['Women', 'Men']
})).mark_text(
    align='left',
    dx=-300,
    dy=-5
).encode(
    y='y:Q',
    text='label:N'
)

labels_left = alt.Chart(pd.DataFrame({
    'y': [0, 1],
    'label': [ 'Men','Women']
})).mark_text(
    align='left',
    dx=-80,
    dy=-5
).encode(
    y='y:Q',
    text='label:N'
)

click = alt.selection_point(
    fields=['preusuel', 'annais'],
    on='click',
    empty=False
)

# lines = alt.Chart(df_pct).mark_line(point=True).encode(
#     x=alt.X('annais:O', title='Year', axis=alt.Axis(values=years_to_show)),
#     y=alt.Y('pct_male:Q', scale=alt.Scale(domain=[0, 1]), title='Proportion male (1) / female (0)'),
#     color='preusuel:N',
#     tooltip=['preusuel', 'annais', alt.Tooltip('pct_male:Q', format='.2f')]
# ).transform_filter(
#     alt.datum.preusuel == name_select
# ).add_params(
#     name_select, click
# )


lines = alt.Chart(df_pct).mark_line(point=True).encode(
    x=alt.X('annais:O', title='Year', axis=alt.Axis(values=years_to_show)),
    y=alt.Y('pct_male:Q', scale=alt.Scale(domain=[0, 1])),
    color='preusuel:N',
    tooltip=['preusuel', 'annais', alt.Tooltip('pct_male:Q', format='.2f')]
).transform_filter(
    click_name
).add_params(
    click_name
)


highlight = alt.Chart(df_pct).mark_point(filled=True, size=150, color='red').encode(
    x='annais:O',
    y='pct_male:Q',
    tooltip=['preusuel', 'annais', alt.Tooltip('pct_male:Q', format='.2f')]
).transform_filter(
    click
).add_params(
    click
)



bar_data = alt.Chart(df_pct).transform_filter(
    click
).transform_calculate(
    prop_female='1 - datum.pct_male'
).transform_fold(
    ['prop_female', 'pct_male'], 
    as_=['gender', 'value']
).encode(
    y=alt.Y('value:Q', stack='zero', scale=alt.Scale(domain=[0, 1])),  
    x=alt.value(20),
    color=alt.Color('gender:N',
                    scale=alt.Scale(domain=['pct_male', 'prop_female'], range=['purple', 'red']),
                    legend=None),
    order=alt.Order(
        'gender:N',
        sort='ascending'  # female first (bottom), male second (top)
    )
).mark_bar().properties(
    width=30,
    height=400,
    title='Ratio (W/M)'
).add_params(
    click
)



rule = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(
    color='grey'
).encode(
    y='y:Q'
)


# chart2 = alt.Chart(df_plot).mark_bar().encode(
#     x='annais:O',
#     y='total:Q',
#     color='sex:N',
#     tooltip=['preusuel', 'total']
# ).add_params(name_select).encode(
#     opacity=alt.condition(name_select, alt.value(1), alt.value(0.5))
# )

chart2 = (lines + highlight + rule + labels ).properties(
    width=500,
    height=400,
    title='Gender proportion over time for selected names'
)
from IPython.display import display, HTML

display(HTML('''
<style>
input[type="range"] {
    width: 400px !important; 
    height: 25px !important; 
}
</style>
'''))

final_chart = alt.hconcat(
    chart, bar_data,
    chart2
).resolve_scale(
    color='independent'
)

final_chart

final_chart.save('Vis3.html')

# fix issue point stays when changing name
# add zoom
# add drop down menu

NameError: name 'pd' is not defined

In [ ]:

# years_to_show = list(range(1900, 2021, 5))

# name_select = alt.param(
#     bind=alt.binding_select(
#         options=sorted(df_pct['preusuel'].unique().tolist()),
#         name='Name: '
#     ),
#     value='CAMILLE'
# )

# click = alt.selection_single(
#     fields=['preusuel', 'annais'],
#     on='click',
#     empty='none'
# )

# labels = alt.Chart(pd.DataFrame({
#     'y': [0, 1],
#     'label': ['Women', 'Men']
# })).mark_text(
#     align='left',
#     dx=-400,
#     dy=-5
# ).encode(
#     y='y:Q',
#     text='label:N'
# )

# labels_left = alt.Chart(pd.DataFrame({
#     'y': [0, 1],
#     'label': [ 'Men','Women']
# })).mark_text(
#     align='left',
#     dx=-80,
#     dy=-5
# ).encode(
#     y='y:Q',
#     text='label:N'
# )
# lines = alt.Chart(df_pct).mark_line(point=True).encode(
#     x=alt.X('annais:O', title='Year', axis=alt.Axis(values=years_to_show)),
#     y=alt.Y('pct_male:Q', scale=alt.Scale(domain=[0, 1]), title='Proportion male (1) / female (0)'),
#     color='preusuel:N',
#     tooltip=['preusuel', 'annais', alt.Tooltip('pct_male:Q', format='.2f')]
# ).transform_filter(
#     alt.datum.preusuel == name_select
# ).add_params(
#     name_select, click
# )


# highlight = alt.Chart(df_pct).mark_point(filled=True, size=150, color='red').encode(
#     x='annais:O',
#     y='pct_male:Q',
#     tooltip=['preusuel', 'annais', alt.Tooltip('pct_male:Q', format='.2f')]
# ).transform_filter(
#     click
# )

# bar_data = alt.Chart(df_pct).transform_filter(
#     click
# ).transform_calculate(
#     prop_female='1 - datum.pct_male'
# ).transform_fold(
#     ['prop_female', 'pct_male'], 
#     as_=['gender', 'value']
# ).encode(
#     y=alt.Y('value:Q', stack='zero', scale=alt.Scale(domain=[0, 1])),  
#     x=alt.value(50),
#     color=alt.Color('gender:N',
#                     scale=alt.Scale(domain=['pct_male', 'prop_female'], range=['blue', 'red']),
#                     legend=None),
#     order=alt.Order(
#         'gender:N',
#         sort='ascending'  # female first (bottom), male second (top)
#     )
# ).mark_bar().properties(
#     width=50,
#     height=400,
#     title='Ratio (Female/Male)'
# )



# rule = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(
#     color='grey'
# ).encode(
#     y='y:Q'
# )


# chart = (lines + highlight + rule + labels).properties(
#     width=700,
#     height=400,
#     title='Gender proportion over time for selected names'
# )

# final_chart = alt.hconcat(
#     bar_data+labels_left,
#     chart
# ).resolve_scale(
#     color='independent'
# )

# final_chart
